<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [169]:
import folium
import pandas as pd
import pickle
import io
import requests
import datetime
import math
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# get mta_stations.csv
mta_stations_csv_path = '../data/mta_stations.csv'

# Get team-wide dataframe of Brooklyn stations
foodtruck_pickle = '../pickle_jar/foodtruck_df2.pkl'

In [35]:
ft_df = pd.read_pickle(foodtruck_pickle)
ft_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,HOLIDAY,ENTRIES,EXITS,LAST_ENTRIES,LAST_EXITS,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK
0,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/01/2018,09:00:00,REGULAR,Saturday,...,False,6293248,4614702,6293557,4615002,13:00:00,309,300,609,True
1,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/02/2018,09:00:00,REGULAR,Sunday,...,False,6294460,4615867,6294784,4616103,13:00:00,324,236,560,True
2,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/03/2018,09:00:00,REGULAR,Monday,...,True,6295556,4616999,6295911,4617273,13:00:00,355,274,629,True
3,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/04/2018,09:00:00,REGULAR,Tuesday,...,False,6297760,4618510,6298607,4618758,13:00:00,847,248,1095,True
4,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/05/2018,09:00:00,RECOVR AUD,Wednesday,...,False,6300702,4620614,6301626,4620892,13:00:00,924,278,1202,True


In [36]:
mta_stations_df = pd.read_csv(mta_stations_csv_path)
mta_stations_df.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan


In [63]:
ft_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,HOLIDAY,ENTRIES,EXITS,LAST_ENTRIES,LAST_EXITS,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK
0,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/01/2018,09:00:00,REGULAR,Saturday,...,False,6293248,4614702,6293557,4615002,13:00:00,309,300,609,True
1,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/02/2018,09:00:00,REGULAR,Sunday,...,False,6294460,4615867,6294784,4616103,13:00:00,324,236,560,True
2,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/03/2018,09:00:00,REGULAR,Monday,...,True,6295556,4616999,6295911,4617273,13:00:00,355,274,629,True
3,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/04/2018,09:00:00,REGULAR,Tuesday,...,False,6297760,4618510,6298607,4618758,13:00:00,847,248,1095,True
4,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/05/2018,09:00:00,RECOVR AUD,Wednesday,...,False,6300702,4620614,6301626,4620892,13:00:00,924,278,1202,True


In [37]:
brooklyn_stations = {'7 AV BQ': [], 'PARK PLACE S': [], 'BOTANIC GARDEN S2345': [], 'PROSPECT PARK BQS': ['11226'], 'PARKSIDE AV BQ': ['11226'], 'CHURCH AV BQ': ['11226'], 'BEVERLEY ROAD BQ': [], 'CORTELYOU RD BQ': ['11226'], 'NEWKIRK PLAZA BQ': ['11226'], 'AVENUE H BQ': ['11230'], 'AVENUE J BQ': ['11230'], 'AVENUE M BQ': ['11230'], 'KINGS HWY BQ': ['11229'], 'AVENUE U BQ': [], 'NECK RD BQ': ['11229'], 'SHEEPSHEAD BAY BQ': ['11235'], 'BRIGHTON BEACH BQ': ['11235'], 'OCEAN PKWY Q': ['11235'], 'BOROUGH HALL R2345': ['11201'], 'JAY ST-METROTEC R': [], 'DEKALB AV BDNQR': ['11217'], 'ATL AV-BARCLAY BDNQR2345': ['11217'], 'UNION ST R': ['11215'], '4AV-9 ST DFGMNR': ['11231'], 'PROSPECT AV R': ['11215'], '25 ST R': ['11232'], '36 ST DNR': ['10012'], '45 ST R': ['11220'], '53 ST R': ['11220'], '59 ST NRW': [], 'BAY RIDGE AV R': ['11220'], '77 ST R': ['11209'], '86 ST R': ['11209'], 'BAY RIDGE-95 ST R': ['11209'], '8 AV N': ['11220'], 'FT HAMILTON PKY N': [], 'NEW UTRECHT AV ND': [], '18 AV N': [], '20 AV N': ['11214'], 'BAY PKWY N': [], 'KINGS HWY N': [], 'AVENUE U N': ['11223'], '86 ST N': ['10028'], '9 AV D': ['11219'], 'FT HAMILTON PKY D': [], '50 ST D': [], '55 ST D': ['11219'], '71 ST D': ['11228'], '79 ST D': ['11220'], '18 AV D': [], '20 AV D': ['11214'], 'BAY PKWY D': ['11214'], '25 AV D': ['11214'], 'BAY 50 ST D': ['11214'], 'CONEY IS-STILLW DFNQ': [], 'W 8 ST-AQUARIUM FQ': ['11224'], 'BEDFORD AV L': ['11211'], 'LORIMER ST LG': ['11211'], 'GRAHAM AV L': ['11211'], 'GRAND ST L': ['11211'], 'MONTROSE AV L': ['11206'], 'MORGAN AV L': ['11237'], 'JEFFERSON ST L': ['11237'], 'DEKALB AV L': ['11237'], 'MYRTLE-WYCKOFF LM': ['11237'], 'HALSEY ST L': [], 'WILSON AV L': ['11207'], 'BUSHWICK AV L': [], 'ATLANTIC AV L': ['11207'], 'SUTTER AV L': ['11207'], 'LIVONIA AV L': ['11212'], 'NEW LOTS L': [], 'EAST 105 ST L': ['11236'], 'CANARSIE-ROCKAW L': ['11236'], 'MARCY AV JMZ': ['11211'], 'HEWES ST JM': ['11211'], 'LORIMER ST JM': [], 'FLUSHING AV JM': ['11206'], 'MYRTLE AV JMZ': [], 'KOSCIUSZKO ST J': [], 'GATES AV JZ': ['11221'], 'HALSEY ST J': [], 'CHAUNCEY ST JZ': ['11207'], 'ALABAMA AV J': ['11207'], 'VAN SICLEN AV JZ': [], 'CLEVELAND ST J': [], 'NORWOOD AV JZ': ['11208'], 'CRESCENT ST JZ': ['11208'], 'CYPRESS HILLS J': [], 'CENTRAL AV M': ['11221'], 'KNICKERBOCKER M': ['11237'], 'HIGH ST AC': ['11201'], 'JAY ST-METROTEC ACF': ['11201'], 'HOYT-SCHER ACG': ['11201'], 'LAFAYETTE AV C': ['11217'], 'CLINTON-WASH AV C': [], 'FRANKLIN AV ACS': ['11238'], 'NOSTRAND AV AC': [], 'KINGSTON-THROOP C': ['11213'], 'UTICA AV AC': ['11233'], 'RALPH AV C': ['11233'], 'ROCKAWAY AV C': ['11233'], 'BROADWAY JCT ACJLZ': [], 'LIBERTY AV C': ['11207'], 'VAN SICLEN AVE C': ['11207'], 'SHEPHERD AV C': ['11208'], 'EUCLID AV AC': ['11208'], 'GREENPOINT AV G': ['11222'], 'NASSAU AV G': ['11222'], 'METROPOLITAN AV GL': [], 'BROADWAY G': ['11206'], 'FLUSHING AV G': ['11206'], 'MYRTLE-WILLOUGH G': ['11206'], 'BEDFORD-NOSTRAN G': ['11205'], 'CLASSON AV G': ['11238'], 'CLINTON-WASH AV G': [], 'FULTON ST G': ['11217'], 'YORK ST F': ['11201'], 'BERGEN ST FG': ['11201'], 'CARROLL ST FG': ['11231'], 'SMITH-9 ST FG': ['11231'], '4 AV-9 ST DFGMNR': ['10014'], '7 AV FG': ['11215'], '15 ST-PROSPECT FG': ['11215'], 'FT HAMILTON PKY FG': [], 'CHURCH AV FG': [], 'DITMAS AV F': ['11218'], '18 AV F': ['11230'], 'AVENUE I F': [], 'BAY PKWY F': ['11230'], 'AVENUE N F': [], 'AVENUE P F': [], 'KINGS HWY F': ['11229'], 'AVENUE U F': [], 'AVENUE X F': ['11223'], 'NEPTUNE AV F': ['11224'], 'CLARK ST 23': [], 'BOROUGH HALL 2345R': ['11201'], 'HOYT ST 23': [], 'NEVINS ST 2345': ['11217'], 'ATL AV-BARCLAY 2345BDNQR': ['11217'], 'BERGEN ST 23': [], 'GRAND ARMY PLAZ 23': [], 'EASTN PKWY-MUSM 23': ['11238'], 'FRANKLIN AV 2345S': ['11238'], 'NOSTRAND AV 3': ['11225'], 'KINGSTON AV 3': ['11213'], 'CROWN HTS-UTICA 34': [], 'SUTTER AV-RUTLD 3': [], 'SARATOGA AV 3': ['11212'], 'ROCKAWAY AV 3': [], 'JUNIUS ST 3': ['11212'], 'PENNSYLVANIA AV 3': ['11207'], 'VAN SICLEN AV 3': ['11207'], 'NEW LOTS AV 3': ['11207'], 'PRESIDENT ST 25': ['11225'], 'STERLING ST 25': ['11225'], 'WINTHROP ST 25': ['11225'], 'CHURCH AV 25': [], 'BEVERLY RD 25': ['11226'], 'NEWKIRK AV 25': [], 'FLATBUSH AV-B.C 25': ['11226']}

In [38]:
for k,v in brooklyn_stations.items():
    if len(v) != 0:
        brooklyn_stations[k] = v[0]
    else:
        brooklyn_stations[k] = ''


In [39]:
brooklyn_stations_zips = pd.DataFrame.from_dict(brooklyn_stations, orient='index')
brooklyn_stations_zips.rename(columns = {0:'zip'}, inplace=True)

In [41]:
brooklyn_stations_zips.sample(5)

,zip
SUTTER AV L,11207
NEVINS ST 2345,11217
15 ST-PROSPECT FG,11215
4AV-9 ST DFGMNR,11231
DITMAS AV F,11218


In [43]:
brooklyn_stations_zips = brooklyn_stations_zips.reset_index()
brooklyn_stations_zips

,index,zip
0,7 AV BQ,
1,PARK PLACE S,
2,BOTANIC GARDEN S2345,
3,PROSPECT PARK BQS,11226
4,PARKSIDE AV BQ,11226
5,CHURCH AV BQ,11226
6,BEVERLEY ROAD BQ,
7,CORTELYOU RD BQ,11226
8,NEWKIRK PLAZA BQ,11226
9,AVENUE H BQ,11230


In [44]:
ft_df_zips = pd.merge(ft_df, brooklyn_stations_zips, left_on=['UNIQUE_STATION'], right_on='index')

In [74]:
ft_df_zips.sample(10)

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,EXITS,LAST_ENTRIES,LAST_EXITS,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK,index,zip
62803,N127,R442,00-00-00,SHEPHERD AV,C,SHEPHERD AV C,11/30/2018,08:00:00,REGULAR,Friday,...,9099540,7653460,9099645,12:00:00,406,105,511,True,SHEPHERD AV C,11208
72047,N535,R220,00-00-02,CARROLL ST,FG,CARROLL ST FG,09/28/2018,08:00:00,REGULAR,Friday,...,3272496,16229937,3272576,12:00:00,1037,80,1117,True,CARROLL ST FG,11231
53588,N103,R127,00-00-00,JAY ST-METROTEC,ACF,JAY ST-METROTEC ACF,11/06/2018,07:00:00,REGULAR,Tuesday,...,5997957,1551144,5998353,11:00:00,86,396,482,True,JAY ST-METROTEC ACF,11201
360,B004,R171,00-00-03,7 AV,BQ,7 AV BQ,11/27/2018,08:00:00,REGULAR,Tuesday,...,2975869,3478665,2976231,12:00:00,1009,362,1371,True,7 AV BQ,
4493,B020,R263,00-00-00,AVENUE H,BQ,AVENUE H BQ,10/30/2018,08:00:00,REGULAR,Tuesday,...,249219,598471,249402,12:00:00,196,183,379,True,AVENUE H BQ,11230
92495,R626,R062,00-05-00,CROWN HTS-UTICA,34,CROWN HTS-UTICA 34,10/18/2018,08:00:00,REGULAR,Thursday,...,0,5785,0,12:00:00,0,0,0,True,CROWN HTS-UTICA 34,
46306,J002,R460,00-05-01,MARCY AV,JMZ,MARCY AV JMZ,09/09/2018,09:00:00,REGULAR,Sunday,...,338,0,338,13:00:00,0,0,0,True,MARCY AV JMZ,11211
9788,B025,R150,00-05-00,AVENUE U,BQ,AVENUE U BQ,11/29/2018,08:00:00,REGULAR,Thursday,...,0,100663468,0,12:00:00,0,0,0,True,AVENUE U BQ,
59514,N120,R153,00-00-01,UTICA AV,AC,UTICA AV AC,11/17/2018,08:00:00,REGULAR,Saturday,...,6124575,5207690,6124818,12:00:00,372,243,615,True,UTICA AV AC,11233
15451,C008,R099,00-04-01,DEKALB AV,BDNQR,DEKALB AV BDNQR,09/20/2018,08:00:00,REGULAR,Thursday,...,2082,3,2082,12:00:00,0,0,0,True,DEKALB AV BDNQR,11217


Map zip code values to coordinates.

In [112]:
zip_to_coord_raw = '''ZIP     LAT    LONG
11207  40.670 -73.900
11201  40.700 -73.990
11226  40.650 -73.960
11217  40.680 -73.980
11211  40.710 -73.950
11220  40.640 -74.020
11214  40.600 -74.010
11206  40.700 -73.940
11237  40.700 -73.920
11230  40.620 -73.970
11208  40.670 -73.870
11225  40.660 -73.960
11215  40.670 -73.980
11238  40.680 -73.960
11233  40.680 -73.920
11209  40.620 -74.030
11231  40.680 -74.000
11212  40.660 -73.920
11229  40.600 -73.940
11235  40.580 -73.950
11224  40.580 -73.990
11222  40.730 -73.950
11219  40.630 -74.000
11236  40.640 -73.900
11223  40.600 -73.970
11221  40.690 -73.930
11213  40.670 -73.940
10012  40.726 -73.998
11218  40.650 -73.980
11232  40.640 -74.020
11228  40.620 -74.010
11205  40.700 -73.970
10014  40.740 -74.010
10028  40.780 -73.950'''

split = [line.split() for line in zip_to_coord_raw.split('\n')]
x = zip_to_coord_raw.split()
zip_to_geo_dict = {x[i]: str(float(x[i+1]))+'_'+str(float(x[i+2])) for i in range(3,len(x),3)}
zip_to_geo_dict

# zip_to_coord_df = pd.DataFrame(split)
# zip_to_coord_df.columns = zip_to_coord_df.iloc[0]
# zip_to_coord_df.drop(0,inplace=True)
# zip_to_coord_df.head()

{'11207': '40.67_-73.9',
 '11201': '40.7_-73.99',
 '11226': '40.65_-73.96',
 '11217': '40.68_-73.98',
 '11211': '40.71_-73.95',
 '11220': '40.64_-74.02',
 '11214': '40.6_-74.01',
 '11206': '40.7_-73.94',
 '11237': '40.7_-73.92',
 '11230': '40.62_-73.97',
 '11208': '40.67_-73.87',
 '11225': '40.66_-73.96',
 '11215': '40.67_-73.98',
 '11238': '40.68_-73.96',
 '11233': '40.68_-73.92',
 '11209': '40.62_-74.03',
 '11231': '40.68_-74.0',
 '11212': '40.66_-73.92',
 '11229': '40.6_-73.94',
 '11235': '40.58_-73.95',
 '11224': '40.58_-73.99',
 '11222': '40.73_-73.95',
 '11219': '40.63_-74.0',
 '11236': '40.64_-73.9',
 '11223': '40.6_-73.97',
 '11221': '40.69_-73.93',
 '11213': '40.67_-73.94',
 '10012': '40.726_-73.998',
 '11218': '40.65_-73.98',
 '11232': '40.64_-74.02',
 '11228': '40.62_-74.01',
 '11205': '40.7_-73.97',
 '10014': '40.74_-74.01',
 '10028': '40.78_-73.95'}

In [113]:
brooklyn_stations_zips['COORD'] = brooklyn_stations_zips['zip'][brooklyn_stations_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z])
brooklyn_stations_zips.head()

,index,zip,COORD
0,7 AV BQ,,NaN
1,PARK PLACE S,,NaN
2,BOTANIC GARDEN S2345,,NaN
3,PROSPECT PARK BQS,11226,40.65_-73.96
4,PARKSIDE AV BQ,11226,40.65_-73.96


In [114]:
type((40.65, -73.96))

tuple

In [124]:
ft_df_zips['COORD'] = ft_df_zips.zip[ft_df_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z].split('_'))
ft_df_zips['LAT'] = ft_df_zips.zip[ft_df_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z].split('_')[0])
ft_df_zips['LONG'] = ft_df_zips.zip[ft_df_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z].split('_')[1])

ft_df_zips.sample(10)

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK,index,zip,COORD,LAT,LONG
71483,N534,R220,01-00-01,CARROLL ST,FG,CARROLL ST FG,09/10/2018,08:00:00,REGULAR,Monday,...,12:00:00,504,9,513,True,CARROLL ST FG,11231,"[40.68, -74.0]",40.68,-74.0
30188,E003,R369,00-05-00,FT HAMILTON PKY,D,FT HAMILTON PKY D,11/06/2018,08:00:00,REGULAR,Tuesday,...,12:00:00,63,69,132,True,FT HAMILTON PKY D,,NaN,NaN,NaN
85472,R608,R056,00-03-01,NEVINS ST,2345,NEVINS ST 2345,09/27/2018,08:00:00,REGULAR,Thursday,...,12:00:00,129,145,274,True,NEVINS ST 2345,11217,"[40.68, -73.98]",40.68,-73.98
3801,B018,R184,00-00-01,CORTELYOU RD,BQ,CORTELYOU RD BQ,09/05/2018,08:00:00,REGULAR,Wednesday,...,12:00:00,715,85,800,True,CORTELYOU RD BQ,11226,"[40.65, -73.96]",40.65,-73.96
81215,N561,R271,00-00-00,AVENUE X,F,AVENUE X F,10/17/2018,08:00:00,REGULAR,Wednesday,...,12:00:00,501,49,550,True,AVENUE X F,11223,"[40.6, -73.97]",40.6,-73.97
6254,B021,R228,00-03-00,AVENUE J,BQ,AVENUE J BQ,09/09/2018,08:00:00,REGULAR,Sunday,...,12:00:00,2,43,45,True,AVENUE J BQ,11230,"[40.62, -73.97]",40.62,-73.97
6489,B021,R228,00-03-02,AVENUE J,BQ,AVENUE J BQ,11/01/2018,08:00:00,REGULAR,Thursday,...,12:00:00,135,67,202,True,AVENUE J BQ,11230,"[40.62, -73.97]",40.62,-73.97
42741,H032,R295,00-05-00,WILSON AV,L,WILSON AV L,11/23/2018,07:00:00,REGULAR,Friday,...,11:00:00,0,0,0,True,WILSON AV L,11207,"[40.67, -73.9]",40.67,-73.9
89481,R621,R060,00-03-01,EASTN PKWY-MUSM,23,EASTN PKWY-MUSM 23,10/05/2018,08:00:00,REGULAR,Friday,...,12:00:00,204,51,255,True,EASTN PKWY-MUSM 23,11238,"[40.68, -73.96]",40.68,-73.96
83161,R602,R108,00-00-02,BOROUGH HALL,2345R,BOROUGH HALL 2345R,11/21/2018,07:00:00,REGULAR,Wednesday,...,11:00:00,221,131,352,True,BOROUGH HALL 2345R,11201,"[40.7, -73.99]",40.7,-73.99


In [88]:
ft_df_zips.dtypes

C/A               object
UNIT              object
SCP               object
STATION           object
LINENAME          object
UNIQUE_STATION    object
DATE              object
TIME              object
DESC              object
DAY_OF_WEEK       object
MONTH             object
YEAR               int64
HOLIDAY             bool
ENTRIES            int64
EXITS              int64
LAST_ENTRIES       int64
LAST_EXITS         int64
LAST_TIME         object
DAILY_ENTRIES      int64
DAILY_EXITS        int64
DAILY_TOTAL        int64
IN_BK               bool
index             object
zip               object
COORD             object
dtype: object

In [177]:
ft_df_zips.groupby(['zip','LAT','LONG'])['DAILY_ENTRIES'].sum().max()

2340344

In [218]:
# from : https://towardsdatascience.com/visualizing-data-at-the-zip-code-level-with-folium-d07ac983db20

with open('../data/tl_2010_36_zcta510/nyc_zip_codes_geo.geo.json','r') as jsonFile:
    zip_geo_data = json.load(jsonFile)
tmp = zip_geo_data
    
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['ZCTA5CE10'] in list(ft_df_zips.zip.unique()):
        geozips.append(tmp['features'][i])

geozips[0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [236]:
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips
new_json['features'][0]

{'type': 'Feature',
 'properties': {'STATEFP10': '36',
  'ZCTA5CE10': '11215',
  'GEOID10': '3611215',
  'CLASSFP10': 'B5',
  'MTFCC10': 'G6350',
  'FUNCSTAT10': 'S',
  'ALAND10': 5641351,
  'AWATER10': 54168,
  'INTPTLAT10': '+40.6626879',
  'INTPTLON10': '-073.9867402',
  'PARTFLG10': 'N'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-73.99649, 40.6687],
    [-73.997092, 40.669267],
    [-73.99745, 40.66978],
    [-73.997998, 40.670626],
    [-73.998272, 40.671153],
    [-73.998659, 40.671705],
    [-73.998404, 40.672008],
    [-73.99748, 40.673162],
    [-73.996975, 40.673709],
    [-73.996917, 40.673773],
    [-73.99664, 40.674134],
    [-73.996014, 40.674994],
    [-73.995578, 40.675137],
    [-73.995275, 40.675778],
    [-73.993978, 40.676171],
    [-73.992566, 40.676106],
    [-73.990802, 40.675783],
    [-73.990426, 40.676249],
    [-73.989914, 40.67685],
    [-73.989435, 40.677459],
    [-73.989388, 40.678285],
    [-73.988973, 40.67885],
    [-73.98856, 40.679559],
 

In [238]:
open('../data/update-zip-geojson.json','w').write(
    json.dumps(new_json,  separators=(',',':'), indent=4, sort_keys=True)
)

643260

In [348]:
zip_min = ft_df_zips.groupby(['LAT','LONG'])['DAILY_TOTAL'].sum().min()
zip_max = ft_df_zips.groupby(['LAT','LONG'])['DAILY_TOTAL'].sum().max()

# Basic map setup

folium_map = folium.Map(location=[40.650002, -73.949997],
                        zoom_start=11.5,
                        min_zoom=11,
                        max_zoom=15,
                        control_scale=True,
                        tiles='OpenStreetMap')

for index, value in ft_df_zips.groupby(['zip','LAT','LONG'])['DAILY_TOTAL'].sum().items():
    radius = 3 + 12*(value - zip_min)/(zip_max - zip_min)
    color = "Green"
    zip_mask = ft_df_zips['zip'] == index[0]
    columns = ['UNIQUE_STATION','DAILY_TOTAL']
    output = ft_df_zips[ft_df_zips['zip'] == '11230'].groupby('UNIQUE_STATION')['DAILY_TOTAL'].median().to_string().split('\n')[1:]
    html = '<strong>daily_total sum: ' + str(value) + '</strong></br></br>'
    for line in output:
        l = [el.strip() for el in line.split('  ')[1:]]
        cleaned = []
        for el in l:
            if len(el.strip())>0:
                cleaned += [el]
        html += '<div style="display:inline"><strong>' + cleaned[0] + '</strong>' + '<span> ' + cleaned[1] + '</span>' + '</div></br>'
    folium.CircleMarker(location=(index[1],index[2]),
                        radius=radius,
                        color=color,
                        popup = folium.Popup(html, max_width=300,min_width=300),
                        fill=True).add_to(folium_map)
    
# Run a cell with the name of the map to display it in the window
folium_map

IndexError: list index out of range

In [346]:
output = ft_df_zips[ft_df_zips['zip'] == '11230'].groupby('UNIQUE_STATION')['DAILY_TOTAL'].median().to_string().split('\n')[1:]

In [347]:
print(output)

['18 AV F        152.0', 'AVENUE H BQ     37.0', 'AVENUE J BQ    132.0', 'AVENUE M BQ    220.5', 'BAY PKWY F      98.0']


In [351]:
html = ''
for line in output:
    l = [el.strip() for el in line.split('   ')]
    print(l)
    cleaned = []
    for el in l:
        if len(el.strip())>0:
            cleaned += [el]
    html += '<div style="display:inline"><strong>' + cleaned[0] + '</strong>' + '<span>' + cleaned[1] + '</span>' + '</div>'

['18 AV F', '', '152.0']
['AVENUE H BQ', '37.0']
['AVENUE J BQ', '132.0']
['AVENUE M BQ', '220.5']
['BAY PKWY F', '', '98.0']


In [ ]:
html

In [335]:
zip_mask = ft_df_zips['zip'] == index[0]
columns = ['UNIQUE_STATION','DAILY_TOTAL']
output = ft_df_zips[zip_mask].groupby('UNIQUE_STATION')['DAILY_TOTAL'].median()
output

UNIQUE_STATION
36 ST DNR    553
Name: DAILY_TOTAL, dtype: int64

In [278]:
type(ft_df_zips[ft_df_zips['zip'] == '11230'][['UNIQUE_STATION','DAILY_TOTAL']].sort_values(ascending=False,by='DAILY_TOTAL')[:3])

pandas.core.frame.DataFrame

In [231]:
zip_min = ft_df_zips.groupby(['LAT','LONG'])['DAILY_TOTALS'].sum().min()
zip_max = ft_df_zips.groupby(['LAT','LONG'])['DAILY_ENTRIES'].sum().max()

# Basic map setup

m = folium.Map(location=[40.650002, -73.949997],
                        zoom_start=11.5,
                        min_zoom=11,
                        max_zoom=13,
                        control_scale=True,
                        tiles='Stamen Toner')

In [234]:
nyc_geo = r'../data/update-zip-geojson.json'

m.choropleth(
    geo_data = nyc_geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,
    data = ft_df_zips,
    key_on = 'feature.properties.ZCTA5CE10',
    columns = ['zip','DAILY_ENTRIES'],
    fill_color = '#fff',
    legend_name = 'Legened'
)

# for index, value in ft_df_zips.groupby(['LAT','LONG'])['DAILY_ENTRIES'].sum().items():
#     radius = 3 + 12*(value - zip_min)/(zip_max - zip_min)
#     color = "Green"
#     folium.CircleMarker(location=(index[0],index[1]),
#                         radius=radius,
#                         color=color,
#                         fill=True).add_to(folium_map)
    
# Run a cell with the name of the map to display it in the window
m.LayerControl().add_to(m)
m

ValueError: #fff is not a valid ColorBrewer code